In [43]:
import tensorflow as tf

k = 0
tfrecord_path = 'data.record'
def _parse_function(example_proto): # 解析函数
    dics = {  
        'sample': tf.FixedLenFeature([5], tf.int64),  # 如果不是标量，一定要在这里说明数组的长度
        'label': tf.FixedLenFeature([], tf.int64)
    }
    parsed_example = tf.parse_single_example(example_proto, dics)
    parsed_example['sample'] = tf.cast(parsed_example['sample'], tf.float32)
    parsed_example['label'] = tf.cast(parsed_example['label'], tf.float32)

    return parsed_example

def read_dataset(tfrecord_path = tfrecord_path):
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    new_dataset = dataset.map(_parse_function)
    shuffle_dataset = new_dataset.shuffle(buffer_size=20000) # 打乱顺序
    batch_dataset = shuffle_dataset.batch(2) # 按batch数量输出
    prefetch_dataset = batch_dataset.prefetch(2000) # 数据提前进入队列，速度会快很多
    iterator = prefetch_dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    return next_element

next_element = read_dataset()
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord) # 文件名开始进入文件名队列和内存
    
    if k == 1:
        while not coord.should_stop():
            print('dataset:', sess.run([next_element['sample'], next_element['label']]))
    else:
        try:
            while not coord.should_stop():
                print('dataset:', sess.run([next_element['sample'], next_element['label']]))
        except tf.errors.OutOfRangeError:
            print("queue is empty")
    
    coord.request_stop()
    coord.join(threads)
    


dataset: [array([[8., 3., 7., 5., 1.],
       [3., 5., 7., 8., 7.]], dtype=float32), array([1., 1.], dtype=float32)]
dataset: [array([[5., 0., 2., 4., 9.],
       [0., 1., 0., 3., 0.]], dtype=float32), array([0., 0.], dtype=float32)]
dataset: [array([[3., 5., 2., 7., 7.],
       [0., 3., 4., 2., 5.]], dtype=float32), array([0., 0.], dtype=float32)]
dataset: [array([[2., 0., 0., 5., 8.],
       [5., 2., 7., 9., 9.]], dtype=float32), array([1., 0.], dtype=float32)]
dataset: [array([[0., 4., 7., 7., 3.],
       [4., 8., 8., 3., 8.]], dtype=float32), array([1., 1.], dtype=float32)]
queue is empty
